# Cosine 유사도(linear_kernel)

+ 이번시간에는 Cosine 유사도(linear_kernel)를 배워보겠습니다.
+ Cosine 유사도(linear_kernel)을 이용하여 단어나 문장이 얼마나 유사한지 , 아니면 관련이 없는지를 확인할수 있습니다.
+ 먼저, 샘플 문장을 추려내고 해당 문장들에 대해 TF-IDF 구합니다.
+ 그런 다음, linear_kernel(Cosine 유사도)을 사용해서 각 문장들끼리의 얼마나 가까운지 먼지를 계산하여 수치화합니다.
+ 결과값이 "문장수 X 문장수" 매트릭스 형태로 나오게 되며
+ 각 매트릭스 안에는 문장별로 다른 문장에 대한 유사도 값 들어 있게 되고, 유사도 값이 크면 클수록 비슷한 문장이라고 판단할수 있습니다.
+ 바로, 실습해 보겠습니다.

### cosine similarity
![cosine_similarity](https://raw.githubusercontent.com/gzone2000/TEMP_TEST/master/cosine_similarity.png)

### 학습목차
1. A방송사 댓글 데이터 불러오기
2. 5개 댓글 문장에 대해 TF-IDF 만들고 linear_kernel 이용하여 Cosine 유사도 확인해 보자
 + 첫번째 문장이 어느 다른 문장과 제일 유사도가 높은지 찾아봅시다.

## 1. A방송사 댓글 데이터 불러오기

In [1]:
# 라이브러리 임포트

# pandas read_excel 사용시 필요 라이브러리
!pip install openpyxl

import os
import re
import pandas as pd

In [2]:
# A_comment_train.xlsx A방송사 댓글 다운로드 및 Pandas read_excel() 함수 활용하여 읽어오기
comment = pd.read_excel('https://github.com/gzone2000/TEMP_TEST/raw/master/A_comment_train.xlsx', engine='openpyxl')
comment.tail()

,Unnamed: 0,data,label
246,246,영상F서비스로 간편하게 설치!좋아요\n우리 회사화이팅!,긍정
247,247,모든 업무에서 맡은바 업무에 서 최선을 다하는 모습이 좋습니다! 화이팅 입니다.,긍정
248,248,"사내방송 특성상 최근 이슈화 되거나, 언급이 자주되는 '키워드'를 중심으로 뉴스를 ...",부정
249,249,방송 시간이 너무 길어요.,부정
250,250,"처음 들어보는 말들이 많은데,, 설명이 없어서 힘드네요.",부정


## 2. 5개 댓글 문장에 대해 TF-IDF 만들고 linear_kernel 이용하여 Cosine 유사도 확인해 보자

#### 5개 문장 선택

In [3]:
# 5개 샘플
data_t = comment.head(5)
data_t

,Unnamed: 0,data,label
0,0,재미는 있는데 시간이 짧은게 아쉽네요~,긍정
1,1,"OO 관련 내용은 우리 직원과는 거리가 멀었음, 특히, 사내에 홍보할 내용은 아니라고 봄",부정
2,2,스토리가 너무 딱딱해서 별로였음,부정
3,3,프로그램A 화이팅하세요!!,긍정
4,4,높은 곳에 올라가는 모습이 너무 위험해 보여요.,부정


In [4]:
# null 확인
data_t['data'].isnull().sum()

0

In [5]:
data_t['data'][:5]

0                                재미는 있는데 시간이 짧은게 아쉽네요~
1    OO 관련 내용은 우리 직원과는 거리가 멀었음, 특히, 사내에 홍보할 내용은 아니라고 봄
2                                    스토리가 너무 딱딱해서 별로였음
3                                       프로그램A 화이팅하세요!!
4                           높은 곳에 올라가는 모습이 너무 위험해 보여요.
Name: data, dtype: object

In [6]:
# 5개 댓글 문장에 대해 tf-idf 수행하여 전체 단어 사전 생성 : 28개 단어

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english').fit(data_t['data'])
print(tfidf.vocabulary_)

{'재미는': 21, '있는데': 20, '시간이': 14, '짧은게': 23, '아쉽네요': 16, 'oo': 0, '관련': 3, '내용은': 4, '우리': 18, '직원과는': 22, '거리가': 1, '멀었음': 8, '특히': 24, '사내에': 12, '홍보할': 26, '아니라고': 15, '스토리가': 13, '너무': 5, '딱딱해서': 7, '별로였음': 10, '프로그램a': 25, '화이팅하세요': 27, '높은': 6, '곳에': 2, '올라가는': 17, '모습이': 9, '위험해': 19, '보여요': 11}


In [7]:
# 5개 댓글 문장에 대해 tf-idf 수행하여 5 X 28 형태로 만듬 : 5개문장 X 28개 단어

tfidf_matrix = tfidf.transform(data_t['data']).toarray()
print(tfidf_matrix.shape)
print(tfidf_matrix)

(5, 28)
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.4472136  0.         0.4472136  0.
  0.         0.         0.4472136  0.4472136  0.         0.4472136
  0.         0.         0.         0.        ]
 [0.26726124 0.26726124 0.         0.26726124 0.53452248 0.
  0.         0.         0.26726124 0.         0.         0.
  0.26726124 0.         0.         0.26726124 0.         0.
  0.26726124 0.         0.         0.         0.26726124 0.
  0.26726124 0.         0.26726124 0.        ]
 [0.         0.         0.         0.         0.         0.42224214
  0.         0.52335825 0.         0.         0.52335825 0.
  0.         0.52335825 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        

#### 보기좋게 TF-IDF를 데이터프레임으로 표현하기

In [8]:
# 단어 인덱스로 정렬
sort_tfidf = dict(sorted(tfidf.vocabulary_.items(), key=lambda x : x[1]))
sort_tfidf

{'oo': 0,
 '거리가': 1,
 '곳에': 2,
 '관련': 3,
 '내용은': 4,
 '너무': 5,
 '높은': 6,
 '딱딱해서': 7,
 '멀었음': 8,
 '모습이': 9,
 '별로였음': 10,
 '보여요': 11,
 '사내에': 12,
 '스토리가': 13,
 '시간이': 14,
 '아니라고': 15,
 '아쉽네요': 16,
 '올라가는': 17,
 '우리': 18,
 '위험해': 19,
 '있는데': 20,
 '재미는': 21,
 '직원과는': 22,
 '짧은게': 23,
 '특히': 24,
 '프로그램a': 25,
 '홍보할': 26,
 '화이팅하세요': 27}

In [9]:
# 단어 인덱스순으로 단어 나열
tfidf_sort_word = sort_tfidf.keys()
tfidf_sort_word

dict_keys(['oo', '거리가', '곳에', '관련', '내용은', '너무', '높은', '딱딱해서', '멀었음', '모습이', '별로였음', '보여요', '사내에', '스토리가', '시간이', '아니라고', '아쉽네요', '올라가는', '우리', '위험해', '있는데', '재미는', '직원과는', '짧은게', '특히', '프로그램a', '홍보할', '화이팅하세요'])

In [10]:
# TF-IDF 보기 좋게 데이터프레임형식으로 표현

import pandas as pd

pd.DataFrame(tfidf_matrix, columns=tfidf_sort_word)

,oo,거리가,곳에,관련,내용은,너무,높은,딱딱해서,멀었음,모습이,...,우리,위험해,있는데,재미는,직원과는,짧은게,특히,프로그램a,홍보할,화이팅하세요
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.447214,0.447214,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000
1,0.267261,0.267261,0.000000,0.267261,0.534522,0.000000,0.000000,0.000000,0.267261,0.000000,...,0.267261,0.000000,0.000000,0.000000,0.267261,0.000000,0.267261,0.000000,0.267261,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.422242,0.000000,0.523358,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.707107
4,0.000000,0.000000,0.387757,0.000000,0.000000,0.312840,0.387757,0.000000,0.000000,0.387757,...,0.000000,0.387757,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
# 첫번째 문서 내용 출력
print('첫번째 문서 :', data_t['data'][0])

첫번째 문서 : 재미는 있는데 시간이 짧은게 아쉽네요~


In [12]:
# 첫번째 문서에 대한 TF-IDF 내용 출력
# 첫번째 문서의 단어가 TF-IDF에서 가중치 숫자로 변경됨 확인

pd.DataFrame(tfidf_matrix[:1], columns=tfidf_sort_word)

,oo,거리가,곳에,관련,내용은,너무,높은,딱딱해서,멀었음,모습이,...,우리,위험해,있는데,재미는,직원과는,짧은게,특히,프로그램a,홍보할,화이팅하세요
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.447214,0.447214,0.0,0.447214,0.0,0.0,0.0,0.0


#### linear_kernel 사용하여 유사도 측정하기

In [13]:
# linear_kernel 이용하여 코사인 유사도를 구합니다.
# TF-IDF된 5개 문장에 대해 5문서 x 5문서 매트릭스 형태의 유사도를 만듭니다.
# 즉, 첫번째 문장과 나머지 4 문장과 관계가 가까운지 멀리 있는지등을 파악할수 있다.
# +1에 가까우면 양의 상관관계가 있고 0이면 아무 관계가 없고 , -1이면 음의 상관관계가 있다고 볼수 있음
# 보기가 쉽지 않네요.

from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.13209408],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.13209408, 0.        , 1.        ]])

In [14]:
# 코사인 유사도를 데이터프레임 형식으로 표현하기 
# 3번 문장과 5번 문장과의 10% 유사도 있음 확인

cols = [ str(i) + '문장' for i in range(1,6)]
ind = [ str(i) + '문장' for i in range(1,6)]

pd.DataFrame(cosine_sim, columns=cols, index=ind)

,1문장,2문장,3문장,4문장,5문장
1문장,1.0,0.0,0.000000,0.0,0.000000
2문장,0.0,1.0,0.000000,0.0,0.000000
3문장,0.0,0.0,1.000000,0.0,0.132094
4문장,0.0,0.0,0.000000,1.0,0.000000
5문장,0.0,0.0,0.132094,0.0,1.000000


## 배운 내용 정리
1. Cosine 유사도(linear_kernel) 사용하여 문장간의 유사성을 측정해 보았습니다.
2. Cosine 유사도(linear_kernel) 하기전에 TF-IDF 이용하여 각 문장내의 단어들에 대해 가중치를 부여 했고
3. Cosine 유사도(linear_kernel) 이용하여 각 문장별로 다른 문장과의 유사도를 계산하여 매트릭스 형태로 수치 표현했습니다.
4. 예를 들어, 첫번째 문장을 보면 첫번째 문장과 다른 문장과의 유사성이 수치화 되어 있어, 첫번째 문장과 다른 문장과 밀접한지, 아닌지를 판단할수 있습니다.
5. 그래서, 코사인 유사도를 이용하여 유사도가 높은 몇개의 상품을 추천할수 도 있다.